In [1]:
%pylab inline
plt.style.use("default")

import os 
import sys
import helpers as h
import utils as u
from glob import glob

from scipy.io import loadmat
from scipy.stats import skew, kurtosis

plt.rcParams['figure.figsize'] = (15, 10) 

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
data_root = "/Users/duuta/ppp/data/stringer/live_data/"
data_files = [fname.split("/")[-1] for fname in glob(f"{data_root}natimg2800_M*.mat")]



In [26]:
def clean_unpack(data0):
    """returns clean and zscored signal"""
    resp0, spon0, istim0 = h.unbox(data0)

    resp0 = h.denoise_resp(resp0, spon0)

    resp0 = h.dupSignal(resp0, istim0)

    return resp0, istim0


In [24]:
_, _, istim = h.unbox(data)

In [40]:
(istim == 0).nonzero()[0]

array([1155, 3955])

In [ ]:
data = loadmat( os.path.join(data_root, data_files[0]), simplify_cells=True)

In [27]:
resp, istim0= clean_unpack(data)


In [34]:
np.nonzero(istim0 == 250)

1

In [35]:
resp.shape

(2, 2800, 10103)

In [39]:
data['stim']['resp'].shape

(5880, 10103)

* need to figure out dupSignal function, seems repeats responses are sought and averaged
* need to figure out what happens... NB: in stringer experiment repeat stimulus in alexnet, there are no repeats. 
* these are important to create a 2* N * M stimulus and then compute cvPCA.
* I could pretend this is spontaneous activity and split the signal then procced

# assuming the responses are spontaneous ( in the sense that there are no repeats)


 as

In [5]:
x, y, z = data['med'].T


In [6]:
def points_above_line(a, b, c, point_list):
    above_indices, below_indices = [], []
    
    for i, point in enumerate(zip(point_list[0], point_list[1])):
        x, y = point
        if a*x + b*y + c > 0:
            above_indices.append(i)
        else:
            below_indices.append(i)


    return above_indices, below_indices



In [13]:
point_list = [x, y]


In [11]:
a, b, c = 59/70, 1, -1168.571

In [14]:
above_points, below_points = points_above_line(a, b, c, point_list)

In [15]:
nab = resp[:, :, above_points]
nbl = resp[:, :, below_points]


In [18]:
def compute_cvPCA(resp0):
    ss0 = u.shuff_cvPCA(resp0, nshuff=10, maxcols=2000)
    ss0 = ss0.mean(axis=0)
    ss0 = ss0 / ss0.sum()

    return ss0


In [20]:
nbl.shape

(2, 2800, 8097)

In [19]:
ss0 = compute_cvPCA(nab)

ValueError: n_components=2800 must be between 0 and min(n_samples, n_features)=2006 with svd_solver='full'

In [56]:
ss1 = compute_cvPCA(nbl)

In [57]:
aalpha, _ = u.get_powerlaw(ss0, np.arange(11, 200).astype('int') )
balpha, _ = u.get_powerlaw(ss1, np.arange(11, 200).astype('int'))


In [50]:
aalpha

1.081978663083578

In [58]:
balpha

1.022544387723973